In [11]:
import torch
from torch import nn, optim, Tensor
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
import lightning as L
from random import random, normalvariate
import plotly.express as px

In [7]:
xs = Tensor([[random()] for _ in range(100)])
ys = Tensor([[0.8 * x.item() - 1.2 + normalvariate(0, 0.1)] for x in xs])
xs.shape, ys.shape 

(torch.Size([100, 1]), torch.Size([100, 1]))

In [8]:
px.scatter(x=xs.squeeze(), y=ys.squeeze(), template='plotly_dark').update_traces(marker=dict(size=5)).show()

In [12]:
class LinearRegressionModel(L.LightningModule):
    def __init__(self, learning_rate=0.1):
        super().__init__()
        self.save_hyperparameters()
        
        # Define the linear layer
        self.linear = nn.Linear(1, 1, bias=True)
        
        # Initialize weights and bias (same as your original)
        nn.init.ones_(self.linear.weight)
        nn.init.zeros_(self.linear.bias)
        
        # Store losses for plotting
        self.train_losses = []
        
    def forward(self, x):
        return self.linear(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = F.mse_loss(preds, y)
        
        # Log metrics
        self.log('train_loss', loss, on_step=True, on_epoch=False, prog_bar=True)
        
        # Store loss for manual tracking (optional)
        self.train_losses.append(loss.item())
        
        # Print progress (similar to your original code)
        current_step = self.global_step + 1
        if current_step == 1 or current_step % 10 == 0:
            weight = self.linear.weight.item()
            bias = self.linear.bias.item()
            print(f"Step {current_step}: loss = {loss.item():.4f}, weight = {weight:.4f}, bias = {bias:.4f}")
        
        return loss
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

In [13]:
# Create dataset and dataloader
dataset = TensorDataset(xs, ys)
dataloader = DataLoader(dataset, batch_size=100, shuffle=False)  # Use full batch like your original

In [14]:
# Create model
model = LinearRegressionModel(learning_rate=0.1)

In [15]:
# Create trainer
trainer = L.Trainer(
    max_epochs=200,  # This will give you 200 steps since batch_size = dataset_size
    enable_checkpointing=False,  # Disable checkpointing for this simple example
    logger=False,  # Disable default logging
    enable_progress_bar=False,  # Disable progress bar to match your output style
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [16]:
# Train the model
trainer.fit(model, dataloader)


  | Name   | Type   | Params | Mode 
------------------------------------------
0 | linear | Linear | 2      | train
------------------------------------------
2         Trainable params
0         Non-trainable params
2         Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode
c:\Users\scott\Documents\code\nn-zero-to-hero-tinygrad\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Step 1: loss = 1.7446, weight = 1.0000, bias = 0.0000
Step 10: loss = 0.0452, weight = 0.1774, bias = -0.8337
Step 20: loss = 0.1715, weight = -0.0283, bias = -1.1590
Step 30: loss = 0.0272, weight = 0.3570, bias = -0.9456
Step 40: loss = 0.0343, weight = 0.5189, bias = -0.9462
Step 50: loss = 0.0178, weight = 0.4903, bias = -1.1154
Step 60: loss = 0.0121, weight = 0.5982, bias = -1.1311
Step 70: loss = 0.0113, weight = 0.6923, bias = -1.1342
Step 80: loss = 0.0100, weight = 0.7092, bias = -1.1862
Step 90: loss = 0.0095, weight = 0.7486, bias = -1.1945
Step 100: loss = 0.0095, weight = 0.7715, bias = -1.2011
Step 110: loss = 0.0095, weight = 0.7762, bias = -1.2124
Step 120: loss = 0.0095, weight = 0.7851, bias = -1.2114


`Trainer.fit` stopped: `max_epochs=200` reached.


Step 130: loss = 0.0095, weight = 0.7854, bias = -1.2137
Step 140: loss = 0.0095, weight = 0.7855, bias = -1.2137
Step 150: loss = 0.0095, weight = 0.7853, bias = -1.2130
Step 160: loss = 0.0095, weight = 0.7842, bias = -1.2130
Step 170: loss = 0.0095, weight = 0.7840, bias = -1.2125
Step 180: loss = 0.0095, weight = 0.7835, bias = -1.2125
Step 190: loss = 0.0095, weight = 0.7834, bias = -1.2123
Step 200: loss = 0.0095, weight = 0.7834, bias = -1.2123


In [17]:
# Access final parameters
print(f"\nFinal weight: {model.linear.weight.item():.4f}")
print(f"Final bias: {model.linear.bias.item():.4f}")


Final weight: 0.7833
Final bias: -1.2123


In [18]:
# Plot training loss
fig = px.line(y=model.train_losses, labels={'x': 'Training Step', 'y': 'MSE Loss'}, title="Training Loss over Time")
fig.show()